# Pandas Commands — One per line with purpose
This notebook-style script lists commonly used pandas commands, one per line, each with an inline comment describing its purpose. Cells follow the same `#%% md` and `#%%` structure as other notebooks in this project.


# Setup: imports and sample data

In [ ]:
import pandas as pd  # import pandas main namespace
import numpy as np   # helpful for numerical examples

# Sample data to use with commands
df = pd.DataFrame({
    'A': [1, 2, 3, 4, 5],
    'B': [10.5, 20.1, np.nan, 40.2, 50.3],
    'C': ['x', 'y', 'y', 'x', 'z'],
    'D': pd.date_range('2025-01-01', periods=5, freq='D')
})  # create a sample DataFrame for demonstrations

s = pd.Series([1, 2, 3, 4], name='S')  # create a sample Series


# Creation

In [ ]:
pd.Series([1, 2, 3])  # create a Series from a list
pd.Series({'a': 1, 'b': 2})  # create a Series from a dict (keys become index)
pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})  # create a DataFrame from dict of lists
pd.DataFrame(np.arange(6).reshape(3, 2), columns=['X', 'Y'])  # create a DataFrame from a NumPy array
pd.date_range('2025-01-01', periods=7, freq='D')  # generate a date index for time series
pd.Categorical(['a', 'b', 'a'])  # create a categorical vector


# Inspection and basic info

In [ ]:
df.head()  # first 5 rows preview
df.tail(3)  # last N rows preview
df.sample(2, random_state=0)  # random sample of rows
df.shape  # tuple of (rows, columns)
df.columns  # column labels
df.index  # row index
df.dtypes  # data types of columns
df.info()  # concise summary of DataFrame
df.describe()  # descriptive statistics for numeric columns
df.describe(include='all')  # descriptive stats including non-numeric
df.memory_usage(deep=True)  # memory usage by column
df.nunique()  # count of distinct values per column


# Selection, indexing, assignment

In [ ]:
df['A']  # select a single column as Series
df[['A', 'B']]  # select multiple columns as DataFrame
df.A  # attribute-style column access (not recommended if name conflicts)
df.loc[0]  # label-based row selection (single row by label)
df.iloc[0]  # position-based row selection (single row by position)
df.loc[0, 'B']  # label-based scalar access (row label, column label)
df.iloc[0, 1]  # position-based scalar access (row pos, col pos)
df.loc[1:3, ['A', 'C']]  # slice rows by label and select specific columns
df.iloc[1:4, 0:2]  # slice rows and columns by position
df.at[0, 'A']  # fast scalar access by label
df.iat[0, 0]  # fast scalar access by position
df.assign(E=df['A'] * 2)  # return new DataFrame with added/modified column
df['E'] = df['A'] + 100  # add/overwrite a column in place
df.insert(1, 'A_times2', df['A'] * 2)  # insert a column at given position
df.rename(columns={'A': 'A_renamed'})  # return new DataFrame with renamed columns
df.rename(columns={'A_renamed': 'A'}, inplace=False)  # demonstrate rename without changing original
df.set_index('C')  # return new DataFrame with a column set as index
df.reset_index()  # return new DataFrame with index turned into a column
df.reindex(range(7))  # reindex to new labels (introduces NaNs for missing rows)
df.filter(like='A')  # keep columns whose names contain substring
df.get('B')  # safely get a column (returns None if missing unless default provided)


# Boolean filtering and conditional ops

In [ ]:
df[df['A'] > 2]  # filter rows where condition holds
df.query('A > 2 and C != "y"')  # query using expression string
df[df['B'].between(10, 40, inclusive='both')]  # filter values within range
df['C'].isin(['x', 'z'])  # membership test per element
df.where(df['A'] % 2 == 0)  # keep values where condition True, else set NaN
df.mask(df['A'] % 2 == 0)  # set values to NaN where condition True
np.where(df['A'] > 3, 'high', 'low')  # vectorized conditional selection via NumPy


# Missing data handling

In [ ]:
df.isna()  # boolean mask of missing values (NaN/NaT)
df.notna()  # boolean mask of non-missing values
df['B'].fillna(0)  # fill missing values with scalar
df['B'].fillna(method='ffill')  # forward-fill missing values
df['B'].fillna(df['B'].mean())  # fill missing with column statistic
df.dropna()  # drop rows with any missing values
df.dropna(subset=['B'])  # drop rows where specific column is missing
df.interpolate()  # interpolate numeric missing values


# Sorting and ranking

In [ ]:
df.sort_values(by='B', ascending=True)  # sort rows by column values
df.sort_index()  # sort rows by index labels
df['A'].rank()  # rank values with average method by default


# Value exploration

In [ ]:
df['C'].value_counts()  # frequency counts of unique values
df['C'].value_counts(normalize=True)  # relative frequencies
df['C'].unique()  # unique values as ndarray
df['C'].nunique(dropna=True)  # number of unique values


# Arithmetic, broadcasting, map/apply/transform

In [ ]:
df.select_dtypes(include=np.number) + 1  # add scalar to numeric columns (broadcast)
df.add(10)  # add scalar to all numeric columns using DataFrame method
df.sub(df['A'], axis=0)  # subtract Series aligning on index
df.mul(2)  # multiply all numeric columns by 2
df.div(2)  # divide all numeric columns by 2
df.pow(2)  # element-wise power for numeric columns
df.apply(np.mean)  # apply function column-wise by default
df.apply(np.mean, axis=1)  # apply function row-wise
df.transform(lambda col: col.fillna(col.mean()))  # transform columns preserving shape
df['A'].map({1: 'one', 2: 'two'})  # map values of a Series via dict
df['A'].apply(lambda x: x * 10)  # apply a function element-wise on a Series
df.clip(lower=0)  # trim values below/above thresholds
df.round(1)  # round numeric columns to N decimals
df.corr(numeric_only=True)  # pairwise correlation of numeric columns
df.cov(numeric_only=True)  # covariance matrix of numeric columns


# GroupBy and aggregation

In [ ]:
df.groupby('C')['A'].sum()  # sum of A grouped by C
df.groupby('C', as_index=False)['A'].mean()  # mean as a DataFrame with group labels as columns
df.groupby(['C'])['A'].agg(['mean', 'sum', 'count'])  # multiple aggregations per group
df.groupby('C').agg(A_mean=('A', 'mean'), B_max=('B', 'max'))  # named aggregations
df.groupby('C').transform(lambda x: x.fillna(x.mean()))  # transform to broadcast back to original shape
df.groupby('C').apply(lambda g: g.head(1))  # apply custom function to each group


# Merge, join, concatenate

In [ ]:
left = pd.DataFrame({'key': ['k1','k2','k3'], 'L': [1,2,3]})  # sample left table for joins
right = pd.DataFrame({'key': ['k1','k2','k4'], 'R': [10,20,40]})  # sample right table for joins
pd.merge(left, right, on='key', how='inner')  # database-style join on key columns
pd.merge(left, right, on='key', how='left')  # left join keeps all left rows
pd.merge(left, right, on='key', how='outer')  # outer join keeps union of keys
left.join(right.set_index('key'), on='key')  # join by index of right
pd.concat([left, right], axis=0, ignore_index=True)  # stack rows from multiple DataFrames
pd.concat([left, right], axis=1)  # concatenate side-by-side by index


# Reshaping: pivot, melt, stack/unstack, crosstab

In [ ]:
wide = pd.DataFrame({'id':[1,1,2,2],'var':['v1','v2','v1','v2'],'val':[10,20,30,40]})  # sample long table
wide.pivot(index='id', columns='var', values='val')  # make columns out of unique values
pd.pivot_table(wide, index='id', columns='var', values='val', aggfunc='mean')  # pivot with aggregation
pd.melt(wide.pivot(index='id', columns='var', values='val').reset_index(), id_vars='id', var_name='var', value_name='val')  # unpivot columns into rows
wide.set_index(['id','var']).unstack('var')  # reshape with MultiIndex to columns
pd.crosstab(wide['id'], wide['var'])  # contingency table of counts


# Window operations

In [ ]:
df['A'].rolling(window=3).mean()  # moving average over rolling window
df['A'].expanding().sum()  # cumulative sum with expanding window
df['A'].ewm(alpha=0.3).mean()  # exponentially weighted moving average


# Date and time handling

In [ ]:
pd.to_datetime(['2025-01-01', '2025-01-05'])  # convert strings to datetime dtype
pd.date_range(start='2025-01-01', end='2025-01-10', freq='2D')  # create a date range with frequency
pd.period_range('2025-01', periods=3, freq='M')  # create a period range (monthly)
df['D'].dt.year  # extract year from datetime column
df['D'].dt.month  # extract month from datetime column
df['D'].dt.day_name()  # get weekday name for each date
df.set_index('D').resample('2D').mean(numeric_only=True)  # resample time series by frequency


# Categorical data

In [ ]:
df['C'].astype('category')  # convert to categorical dtype
df['C'].astype('category').cat.categories  # list distinct categories
df['C'].astype('category').cat.codes  # integer codes for categories
df['C'].astype('category').cat.add_categories(['new'])  # add new categories


# String methods (vectorized)

In [ ]:
df['C'].str.upper()  # uppercase strings
df['C'].str.contains('x')  # check substring presence per element
df['C'].str.replace('x', 'X', regex=False)  # replace substring (non-regex)
df['C'].str.len()  # length of each string
df['C'].str.slice(0, 1)  # slice each string


# Duplicates, reshuffling, and utilities

In [ ]:
df.duplicated()  # boolean mask of duplicated rows
df.drop_duplicates()  # remove duplicate rows
df.sample(frac=0.4, random_state=42)  # random fraction of rows
df.take([0, 2, 4])  # take rows by positional indices
df.pipe(lambda _df: _df.head(2))  # functional-style method chaining via pipe


# Type conversion and replacements

In [ ]:
df.astype({'A': 'float64'})  # convert column(s) to specific dtype
pd.to_numeric(pd.Series(['1', '2', 'x']), errors='coerce')  # convert to numeric, set invalid parsing as NaN
df.replace({np.nan: -1})  # replace values, including NaN handling
df.clip(lower=0, upper=100)  # constrain values within bounds


# Evaluation and expressions

In [ ]:
df.eval('A_plus_B = A + B')  # evaluate string expressions to create/modify columns
df.query('A % 2 == 1')  # select rows using expression language


# MultiIndex operations

In [ ]:
mi = pd.MultiIndex.from_product([['g1','g2'], ['x','y']], names=['grp','key'])  # create a MultiIndex
mdf = pd.DataFrame({'val':[1,2,3,4]}, index=mi)  # DataFrame with MultiIndex
mdf.unstack('key')  # move specified level from index to columns
mdf.stack()  # inverse of unstack: move columns back into index
mdf.swaplevel('grp','key')  # swap two index levels
mdf.sort_index(level='grp')  # sort by a specific index level


# Input/Output (examples)
# Note: I/O commands are shown as examples; uncomment and adjust paths before using.

In [ ]:
# pd.read_csv('path/to/file.csv')  # read CSV from file
# pd.read_excel('path/to/file.xlsx', sheet_name=0)  # read Excel file
# pd.read_json('path/to/file.json')  # read JSON file
# pd.read_parquet('path/to/file.parquet')  # read Parquet file (requires pyarrow or fastparquet)
# pd.read_feather('path/to/file.feather')  # read Feather file
# pd.read_pickle('path/to/file.pkl')  # read Python pickle
# pd.read_html('https://example.com/tablepage')  # read HTML tables into list of DataFrames
# pd.read_sql('SELECT * FROM table', con=some_sqlalchemy_engine)  # read from SQL query

# df.to_csv('out.csv', index=False)  # write DataFrame to CSV
# df.to_excel('out.xlsx', index=False)  # write DataFrame to Excel
# df.to_json('out.json', orient='records')  # write DataFrame to JSON
# df.to_parquet('out.parquet', index=False)  # write DataFrame to Parquet
# df.to_feather('out.feather')  # write DataFrame to Feather
# df.to_pickle('out.pkl')  # write DataFrame to pickle


# Plotting (requires matplotlib)

In [ ]:
df.plot(y='B', kind='line')  # quick line plot of column B
df.plot.scatter(x='A', y='B')  # scatter plot of A vs B
df['A'].plot.hist(bins=5)  # histogram of A


# Options and display settings

In [ ]:
pd.set_option('display.max_rows', 100)  # set max rows to display
pd.set_option('display.max_columns', 20)  # set max columns to display
pd.reset_option('display.max_rows')  # reset an option to default
pd.options.display.float_format = '{:,.2f}'.format  # set global float display format


# Performance and memory tips

In [ ]:
df.astype({'A': 'int32', 'B': 'float32'})  # downcast dtypes to reduce memory
df['C'].astype('category')  # use categorical dtype for repeated strings
pd.eval('x + y', engine='numexpr', parser='pandas', target=None, local_dict={'x': df['A'], 'y': df['A']})  # fast eval of expressions
df.memory_usage(deep=True).sum()  # total memory usage in bytes
